# 🧝‍♂️ The Elves' Guide to Dynamic Pricing Magic ✨

---

**Welcome, dear apprentice!** Step into the enchanted world of the Elves' Marketplace, where pricing isn't just numbers on a tag—it's **pure magic**! 🎩✨

## 🏰 What is This Mystical Art?

Imagine if prices could dance and change like the Northern Lights, responding to the rhythm of demand, the whispers of the seasons, and the ebb and flow of our treasure chests (inventory). This is **Dynamic Pricing**—the ancient elven art of finding the "*just right*" price for every moment!

### 🎯 The Quest Ahead

In this magical journey, you'll discover:
- 📊 **The Elves' Marketplace Ledger**: Explore our treasure trove of sales data
- 🧙‍♂️ **Pricing Spells & Potions**: Learn the fundamental concepts
- ⚗️ **Crafting Price Potions**: Build magical algorithms that adjust prices
- 💰 **Counting the Gold**: Evaluate which strategies bring the most treasure
- 🌟 **Wisdom for Your Own Kingdom**: Take these secrets back to your realm

### 🎭 Meet Your Guide

I'm **Pricewinkle the Wise**, your friendly elf guide! I've been managing pricing magic in our marketplace for over 200 seasons. Don't worry if you're new to this—I'll explain everything as if you're my favorite apprentice! 🧝‍♂️📚

---

*Ready to begin? Let's cast our first spell by importing the magical libraries we'll need!* ✨

In [ ]:
# 🧙‍♂️ Importing Our Magical Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
from datetime import datetime, timedelta

# 🎨 Making our plots beautiful
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

print("✨ All magical libraries imported successfully! ✨")
print("🧝‍♂️ Pricewinkle says: 'We're ready to explore the data!'")

# 📚 The Elves' Marketplace: Our Treasure Trove of Data

---

Welcome to our **Marketplace Ledger**! This ancient tome contains every transaction from our enchanted marketplace over the past year. Each entry tells a story of supply, demand, and the delicate dance of pricing magic! 📖✨

In [ ]:
# 📖 Loading the Elves' Marketplace Ledger
print("🧝‍♂️ Pricewinkle: 'Let me open our ancient ledger for you...'")

# Load the data
df = pd.read_csv('elves_marketplace_data.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"📊 Our ledger contains {len(df):,} magical transactions!")
print(f"🗓️ From {df['timestamp'].min().date()} to {df['timestamp'].max().date()}")
print(f"🛍️ Featuring {df['product_id'].nunique()} unique products")
print(f"👥 Served {df['customer_id'].nunique()} valued customers")

print("\n✨ Let's peek at the first few entries in our ledger:")
display(df.head(10))

In [ ]:
# 🔍 A Closer Look at Our Data
print("🧝‍♂️ Pricewinkle: 'Let me show you the structure of our magical data!'")
print("\n📋 Data Information:")
df.info()

print("\n📊 Statistical Summary:")
display(df.describe())

print("\n🏪 Our Product Categories:")
category_counts = df['category'].value_counts()
for category, count in category_counts.items():
    print(f"  {category}: {count:,} transactions")

In [ ]:
# 🎮 Interactive Data Explorer
print("🧝‍♂️ Pricewinkle: 'Try these magical controls to explore our data!'")

# Create interactive widgets
category_dropdown = widgets.Dropdown(
    options=['All'] + list(df['category'].unique()),
    value='All',
    description='Category:',
    style={'description_width': 'initial'}
)

price_slider = widgets.FloatRangeSlider(
    value=[df['price_paid'].min(), df['price_paid'].max()],
    min=df['price_paid'].min(),
    max=df['price_paid'].max(),
    step=1.0,
    description='Price Range:',
    style={'description_width': 'initial'}
)

def update_data_view(category, price_range):
    filtered_df = df.copy()
    
    if category != 'All':
        filtered_df = filtered_df[filtered_df['category'] == category]
    
    filtered_df = filtered_df[
        (filtered_df['price_paid'] >= price_range[0]) & 
        (filtered_df['price_paid'] <= price_range[1])
    ]
    
    print(f"✨ Found {len(filtered_df):,} magical transactions matching your criteria!")
    
    if len(filtered_df) > 0:
        print(f"💰 Average price: {filtered_df['price_paid'].mean():.2f} gold coins")
        print(f"📦 Average quantity: {filtered_df['quantity'].mean():.1f} items")
        display(filtered_df[['product_name', 'category', 'price_paid', 'quantity', 'timestamp']].head())

# Create interactive widget
interactive_explorer = widgets.interactive(update_data_view, category=category_dropdown, price_range=price_slider)
display(interactive_explorer)

In [ ]:
# 📊 Visual Stories from Our Data
print("🧝‍♂️ Pricewinkle: 'Pictures tell the best stories! Let me show you...'")

# Create subplots for comprehensive view
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🎨 The Elves\' Marketplace in Pictures', fontsize=16, fontweight='bold')

# 1. Price distribution
axes[0,0].hist(df['price_paid'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].set_title('💰 Distribution of Prices (Gold Coins)')
axes[0,0].set_xlabel('Price Paid')
axes[0,0].set_ylabel('Number of Transactions')

# 2. Sales by category
category_sales = df.groupby('category')['quantity'].sum().sort_values(ascending=True)
axes[0,1].barh(category_sales.index, category_sales.values, color='lightgreen')
axes[0,1].set_title('🛍️ Total Items Sold by Category')
axes[0,1].set_xlabel('Total Quantity Sold')

# 3. Revenue over time
df['revenue'] = df['price_paid'] * df['quantity']
monthly_revenue = df.groupby(df['timestamp'].dt.to_period('M'))['revenue'].sum()
axes[1,0].plot(range(len(monthly_revenue)), monthly_revenue.values, marker='o', color='gold', linewidth=2)
axes[1,0].set_title('📈 Monthly Revenue Trends')
axes[1,0].set_xlabel('Month')
axes[1,0].set_ylabel('Revenue (Gold Coins)')
axes[1,0].tick_params(axis='x', rotation=45)

# 4. Price vs Original Price scatter
axes[1,1].scatter(df['original_price'], df['price_paid'], alpha=0.5, color='purple')
axes[1,1].plot([df['original_price'].min(), df['original_price'].max()], 
               [df['original_price'].min(), df['original_price'].max()], 
               'r--', label='Equal Price Line')
axes[1,1].set_title('🎯 Original vs Actual Prices')
axes[1,1].set_xlabel('Original Price')
axes[1,1].set_ylabel('Price Paid')
axes[1,1].legend()

plt.tight_layout()
plt.show()

print("\n✨ What do these magical charts tell us?")
print("💡 Dynamic pricing is already at work - prices paid often differ from original prices!")

# 🧙‍♂️ The Elves' Pricing Spells: Understanding the Magic

---

Now that we've explored our data, let **Pricewinkle the Wise** teach you the fundamental concepts behind pricing magic! ✨

## 🎭 What is Dynamic Pricing?

**Dynamic Pricing** is like having a wise shop assistant who constantly adjusts prices based on:
- 🌟 **Customer Demand**: More people want it? Price goes up!
- 📦 **Inventory Levels**: Running low? Time to increase the price!
- 🗓️ **Time & Seasonality**: Holiday season? Prices dance higher!
- 👥 **Customer Segments**: Loyal customers might get special treatment
- 🏪 **Competition**: What are other shops charging?

## 🎯 The Magic Behind the Curtain

Think of dynamic pricing as **three types of spells**:

### 1. 📚 **Rule-Based Magic** (Simple but Powerful!)
- "*If inventory < 10 items, increase price by 20%*"
- "*If it's a holiday, add 15% premium*"
- "*If demand is high, raise price by 25%*"

### 2. 🧮 **Mathematical Spells** (Using Ancient Algorithms!)
- **Linear Regression**: Finding the invisible thread between price and demand
- **Machine Learning**: Teaching our systems to learn from patterns

### 3. 🧠 **Learning Magic** (Advanced Sorcery!)
- **Reinforcement Learning**: Systems that learn from trial and error
- **Neural Networks**: Digital brains that recognize complex patterns

---

*Ready to brew your first pricing potion? Let's start with some simple magic!* 🧪✨

In [ ]:
# 🧪 Brewing Our First Pricing Potion - Rule-Based Magic!
print("🧝‍♂️ Pricewinkle: 'Let's start with simple but powerful rule-based pricing!'")

def rule_based_pricing(original_price, inventory_level, is_holiday, is_weekend, customer_segment):
    """
    🧙‍♂️ The Elves' Rule-Based Pricing Spell
    
    This magical function applies simple rules to adjust prices:
    - Low inventory = higher prices
    - Holidays = premium pricing  
    - Weekends = slight increase
    - Customer loyalty = discounts
    """
    adjusted_price = original_price
    adjustments = []
    
    # Rule 1: Inventory-based pricing
    if inventory_level < 10:
        multiplier = 1.25  # 25% increase for low stock
        adjusted_price *= multiplier
        adjustments.append(f"📦 Low stock (+{(multiplier-1)*100:.0f}%)")
    elif inventory_level > 80:
        multiplier = 0.95  # 5% discount for high stock
        adjusted_price *= multiplier
        adjustments.append(f"📦 High stock ({(multiplier-1)*100:.0f}%)")
    
    # Rule 2: Holiday premium
    if is_holiday:
        multiplier = 1.20  # 20% holiday premium
        adjusted_price *= multiplier
        adjustments.append(f"🎄 Holiday season (+{(multiplier-1)*100:.0f}%)")
    
    # Rule 3: Weekend premium
    if is_weekend:
        multiplier = 1.10  # 10% weekend premium
        adjusted_price *= multiplier
        adjustments.append(f"🌅 Weekend (+{(multiplier-1)*100:.0f}%)")
    
    # Rule 4: Customer loyalty discount
    if customer_segment == 'Loyal':
        multiplier = 0.90  # 10% loyalty discount
        adjusted_price *= multiplier
        adjustments.append(f"👑 Loyal customer ({(multiplier-1)*100:.0f}%)")
    
    return round(adjusted_price, 2), adjustments

# Test our rule-based pricing
print("\n🧪 Testing Our Rule-Based Pricing Spell:")
print("="*50)

test_cases = [
    (50.0, 5, True, True, 'New'),        # Low stock + Holiday + Weekend + New customer
    (30.0, 90, False, False, 'Loyal'),   # High stock + Regular day + Loyal customer
    (75.0, 50, True, False, 'Regular'),  # Normal stock + Holiday + Regular customer
]

for i, (price, inventory, holiday, weekend, segment) in enumerate(test_cases, 1):
    new_price, adjustments = rule_based_pricing(price, inventory, holiday, weekend, segment)
    print(f"\n📋 Test Case {i}:")
    print(f"   Original Price: {price} gold coins")
    print(f"   Inventory Level: {inventory}")
    print(f"   Holiday: {'Yes' if holiday else 'No'}")
    print(f"   Weekend: {'Yes' if weekend else 'No'}")
    print(f"   Customer: {segment}")
    print(f"   ✨ Final Price: {new_price} gold coins")
    print(f"   📝 Adjustments: {', '.join(adjustments) if adjustments else 'None'}")
    print(f"   📊 Change: {((new_price/price - 1) * 100):+.1f}%")

print("\n🎉 Our rule-based pricing spell is working perfectly!")

In [ ]:
# 🎮 Interactive Rule-Based Pricing Simulator
print("🧝‍♂️ Pricewinkle: 'Now you try! Adjust the magical dials and see prices change!'")

# Create interactive widgets
price_input = widgets.FloatSlider(value=50.0, min=10.0, max=200.0, step=5.0, description='Original Price:')
inventory_input = widgets.IntSlider(value=50, min=0, max=100, description='Inventory:')
holiday_input = widgets.Checkbox(value=False, description='Holiday Season')
weekend_input = widgets.Checkbox(value=False, description='Weekend')
segment_input = widgets.Dropdown(options=['New', 'Regular', 'Loyal', 'High-Value'], value='Regular', description='Customer:')

def interactive_pricing(original_price, inventory, is_holiday, is_weekend, customer_segment):
    new_price, adjustments = rule_based_pricing(original_price, inventory, is_holiday, is_weekend, customer_segment)
    
    print(f"🏷️ Original Price: {original_price:.2f} gold coins")
    print(f"✨ Adjusted Price: {new_price:.2f} gold coins")
    print(f"📊 Total Change: {((new_price/original_price - 1) * 100):+.1f}%")
    
    if adjustments:
        print(f"\n🔮 Applied Spells:")
        for adjustment in adjustments:
            print(f"   • {adjustment}")
    else:
        print("\n😌 No price adjustments needed - standard pricing applies!")
    
    # Visual representation
    if new_price > original_price:
        print(f"\n📈 Price increased by {new_price - original_price:.2f} gold coins")
    elif new_price < original_price:
        print(f"\n📉 Price decreased by {original_price - new_price:.2f} gold coins")
    else:
        print(f"\n➡️ Price remains unchanged")

# Create the interactive widget
pricing_simulator = widgets.interactive(
    interactive_pricing,
    original_price=price_input,
    inventory=inventory_input,
    is_holiday=holiday_input,
    is_weekend=weekend_input,
    customer_segment=segment_input
)

display(pricing_simulator)

In [ ]:
# 🧮 Advanced Magic: Linear Regression Price Optimization
print("🧝‍♂️ Pricewinkle: 'Now for some advanced sorcery - mathematical price prediction!'")

# Prepare data for machine learning
print("📊 Preparing our data for the mathematical spell...")

# Create features for our model
df['price_difference'] = df['price_paid'] - df['original_price']
df['is_weekend'] = df['timestamp'].dt.weekday >= 5
df['month'] = df['timestamp'].dt.month
df['day_of_year'] = df['timestamp'].dt.dayofyear

# Select features for our model
features = [
    'original_price', 'inventory_level_before_sale', 'holiday_season',
    'is_weekend', 'competitor_price_avg', 'month'
]

# Prepare the data
X = df[features].copy()

# Convert boolean to numeric
X['is_weekend'] = X['is_weekend'].astype(int)

# Target variable: quantity sold (demand)
y = df['quantity']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
print("🧙‍♂️ Training our magical demand prediction model...")
demand_model = LinearRegression()
demand_model.fit(X_train, y_train)

# Make predictions
y_pred = demand_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n📊 Model Performance:")
print(f"   R² Score: {r2:.3f} (Higher is better, max 1.0)")
print(f"   Mean Squared Error: {mse:.3f}")

# Show feature importance
print(f"\n🔍 Which factors most influence demand?")
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': np.abs(demand_model.coef_)
}).sort_values('Importance', ascending=False)

for idx, row in feature_importance.iterrows():
    print(f"   {row['Feature']}: {row['Importance']:.3f}")

print("\n✨ Our mathematical spell can predict demand based on various factors!")

In [ ]:
# 🎯 Creating an Intelligent Pricing Function
print("🧝‍♂️ Pricewinkle: 'Let me show you how to use our demand model for smart pricing!'")

def predict_optimal_price(original_price, inventory_level, is_holiday, is_weekend, 
                         competitor_price, month=6, price_steps=20):
    """
    🧙‍♂️ The Elves' Intelligent Pricing Spell
    
    This function uses our trained model to find the price that maximizes revenue!
    """
    
    # Test different price points
    min_price = original_price * 0.7  # 30% discount max
    max_price = original_price * 1.5  # 50% premium max
    
    test_prices = np.linspace(min_price, max_price, price_steps)
    revenues = []
    demands = []
    
    for test_price in test_prices:
        # Create feature vector for prediction
        features_array = np.array([[
            test_price,  # original_price -> test_price
            inventory_level,
            1 if is_holiday else 0,
            1 if is_weekend else 0,
            competitor_price,
            month
        ]])
        
        # Predict demand
        predicted_demand = demand_model.predict(features_array)[0]
        # Ensure demand is positive
        predicted_demand = max(0.1, predicted_demand)
        
        # Calculate revenue
        revenue = test_price * predicted_demand
        
        revenues.append(revenue)
        demands.append(predicted_demand)
    
    # Find optimal price
    optimal_idx = np.argmax(revenues)
    optimal_price = test_prices[optimal_idx]
    optimal_demand = demands[optimal_idx]
    optimal_revenue = revenues[optimal_idx]
    
    return {
        'optimal_price': optimal_price,
        'predicted_demand': optimal_demand,
        'predicted_revenue': optimal_revenue,
        'price_change_pct': ((optimal_price / original_price) - 1) * 100,
        'test_prices': test_prices,
        'revenues': revenues,
        'demands': demands
    }

# Test the intelligent pricing
print("\n🧪 Testing Our Intelligent Pricing Spell:")
print("="*50)

test_scenarios = [
    {'original_price': 50.0, 'inventory_level': 5, 'is_holiday': True, 'is_weekend': True, 'competitor_price': 55.0},
    {'original_price': 30.0, 'inventory_level': 90, 'is_holiday': False, 'is_weekend': False, 'competitor_price': 28.0},
    {'original_price': 100.0, 'inventory_level': 25, 'is_holiday': True, 'is_weekend': False, 'competitor_price': 105.0}
]

for i, scenario in enumerate(test_scenarios, 1):
    result = predict_optimal_price(**scenario)
    
    print(f"\n📋 Scenario {i}:")
    print(f"   Original Price: {scenario['original_price']:.2f} gold coins")
    print(f"   Inventory: {scenario['inventory_level']} items")
    print(f"   Holiday: {'Yes' if scenario['is_holiday'] else 'No'}")
    print(f"   Weekend: {'Yes' if scenario['is_weekend'] else 'No'}")
    print(f"   Competitor Price: {scenario['competitor_price']:.2f} gold coins")
    print(f"   \n🎯 AI Recommendation:")
    print(f"   Optimal Price: {result['optimal_price']:.2f} gold coins")
    print(f"   Expected Demand: {result['predicted_demand']:.1f} items")
    print(f"   Expected Revenue: {result['predicted_revenue']:.2f} gold coins")
    print(f"   Price Change: {result['price_change_pct']:+.1f}%")

print("\n🎉 Our AI-powered pricing spell is working brilliantly!")